In [ ]:
# ==============================
# GAN 기반 이미지 생성 모델 구현 (MNIST/FashionMNIST)
# ==============================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

# -----------------------------
# 1. Generator 정의
# -----------------------------
class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28*28),
            nn.Tanh()   # 출력값 범위: [-1, 1]
        )

    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)

# -----------------------------
# 2. Discriminator 정의
# -----------------------------
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()   # 진짜일 확률 반환
        )

    def forward(self, x):
        return self.model(x)

# -----------------------------
# 3. 데이터 전처리 및 로더
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))   # [-1, 1] 정규화
])

# MNIST 사용 (FashionMNIST으로 바꾸려면 아래 주석 해제)
dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
# dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)

loader = DataLoader(dataset, batch_size=128, shuffle=True)

# -----------------------------
# 4. 모델, 손실 함수, 옵티마이저
# -----------------------------
z_dim = 100
G = Generator(z_dim)
D = Discriminator()

criterion = nn.BCELoss()
opt_G = optim.Adam(G.parameters(), lr=0.0002)
opt_D = optim.Adam(D.parameters(), lr=0.0002)

# -----------------------------
# 5. 학습 루프
# -----------------------------
epochs = 200  # MNIST 기준 권장: 200~300
os.makedirs("samples", exist_ok=True)
fixed_noise = torch.randn(64, z_dim)

for epoch in range(1, epochs+1):
    for real, _ in loader:
        bs = real.size(0)

        # -----------------
        # (1) Discriminator 학습
        # -----------------
        noise = torch.randn(bs, z_dim)
        fake = G(noise)

        D_real = D(real)
        D_fake = D(fake.detach())

        loss_D = criterion(D_real, torch.ones_like(D_real)) + \
                 criterion(D_fake, torch.zeros_like(D_fake))

        opt_D.zero_grad()
        loss_D.backward()
        opt_D.step()

        # -----------------
        # (2) Generator 학습
        # -----------------
        D_fake = D(fake)
        loss_G = criterion(D_fake, torch.ones_like(D_fake))  # 가짜 이미지를 진짜로 속이기

        opt_G.zero_grad()
        loss_G.backward()
        opt_G.step()

    # -----------------
    # 에폭 결과 출력
    # -----------------
    print(f"[에폭 {epoch}/{epochs}] 판별자 손실: {loss_D.item():.4f} | 생성자 손실: {loss_G.item():.4f}")

    # 10 에폭마다 샘플 이미지 저장
    if epoch % 10 == 0:
        with torch.no_grad():
            fake = G(fixed_noise)
            save_image(fake, f"samples/epoch_{epoch}.png", nrow=8, normalize=True)
            print(f">>> 샘플 이미지 저장 완료: samples/epoch_{epoch}.png")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:02<00:00, 3.34MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 138kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.54MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.46MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

[에폭 1/200] 판별자 손실: 0.2047 | 생성자 손실: 5.4901
[에폭 2/200] 판별자 손실: 0.2045 | 생성자 손실: 5.0471
[에폭 3/200] 판별자 손실: 1.2149 | 생성자 손실: 2.4304
[에폭 4/200] 판별자 손실: 0.3515 | 생성자 손실: 3.2370
[에폭 5/200] 판별자 손실: 0.1276 | 생성자 손실: 4.0042
[에폭 6/200] 판별자 손실: 0.1598 | 생성자 손실: 4.5023
[에폭 7/200] 판별자 손실: 0.1207 | 생성자 손실: 4.0480
[에폭 8/200] 판별자 손실: 0.0652 | 생성자 손실: 5.5192
[에폭 9/200] 판별자 손실: 0.0627 | 생성자 손실: 5.4869
[에폭 10/200] 판별자 손실: 0.2141 | 생성자 손실: 7.0564
>>> 샘플 이미지 저장 완료: samples/epoch_10.png
[에폭 11/200] 판별자 손실: 0.4302 | 생성자 손실: 5.8608
[에폭 12/200] 판별자 손실: 0.3180 | 생성자 손실: 3.5773
[에폭 13/200] 판별자 손실: 0.1020 | 생성자 손실: 5.6188
[에폭 14/200] 판별자 손실: 0.1926 | 생성자 손실: 3.0638
[에폭 15/200] 판별자 손실: 0.2016 | 생성자 손실: 3.1447
[에폭 16/200] 판별자 손실: 0.4160 | 생성자 손실: 4.8339
[에폭 17/200] 판별자 손실: 0.1193 | 생성자 손실: 5.5106
[에폭 18/200] 판별자 손실: 0.1855 | 생성자 손실: 4.8531
[에폭 19/200] 판별자 손실: 0.1218 | 생성자 손실: 4.6765
[에폭 20/200] 판별자 손실: 0.2142 | 생성자 손실: 4.8649
>>> 샘플 이미지 저장 완